In [1]:
import numpy as np 
from scipy.stats import multivariate_normal
import Estimateurs

On commence par tirer le $\theta$* ($\in \mathbb{R}$) qui sera le paramètre que l'on cherchera à estimer par la suite. 

A la manière de la génération des données dans l'article Rainforth et al. (2018) : $\theta^* \sim \mathcal{N}(0, 1)$

In [2]:
#theta_true = np.random.multivariate_normal(np.zeros(20), np.identity(20))
theta_true = np.random.normal(0, 1)
print(f"La valeur de theta à estimer est {int(theta_true*100)/100}")

La valeur de theta à estimer est 0.08


Désormais, on va tirer un échantillon $((Z_i, X_i))_i \stackrel{iid}{\sim} \mathcal{N}(\boldsymbol{z}|\theta, \boldsymbol{I}) * \mathcal{N}(\boldsymbol{x}|\boldsymbol{z}, \boldsymbol{I})$. 


In [3]:
## On se donne notre échantillon x

x, _ = Estimateurs.joint_probability(theta_true)

print(f"L'échantillon x observé est :\n \n x = {x} \n \n et sa taille est {x.shape}")

L'échantillon x observé est :
 
 x = [ 0.20694761 -1.77741858  0.49300505 -0.16893673  1.38833637 -1.55210182
  0.6437169  -0.71711827 -0.56588384  3.20037538 -0.49377527 -0.47837875
 -1.72244462  0.9706274   0.69369104 -0.88425392  2.31302701  2.41096425
 -0.1924812   1.12517248] 
 
 et sa taille est (20,)


Ceci nous suffit pour tester notre classe Estimateurs dont le code se trouve dans le fichier Estimateurs.py

In [14]:
theta_hat = np.mean(x)
A, b = Estimateurs.noised_params((1/2)*np.eye(20), (np.zeros(20) + theta_true)/2)
A_hat, b_hat = Estimateurs.noised_params((1/2)*np.eye(20), (np.zeros(20) + theta_hat)/2)

In [17]:
z_sample = Estimateurs.generate_encoder(x, 4, A, b)[0].T

def compute_ratio(x, z, theta, A, b):
    # Parameters
    I = np.eye(20)

    # Probability densities
    p_theta_xz = np.exp(-0.5 * np.dot(np.dot((z - theta).T, np.linalg.inv(2*I)), (z - theta)) - 0.5 * np.dot(np.dot((x - z).T, np.linalg.inv(I)), (x - z)))
    q_phi_z_given_x_density = np.exp(-0.5 * np.dot(np.dot((z - (np.dot(A, x) + b)).T, np.linalg.inv((2/3)*I)), (z - (np.dot(A, x) + b.flatten()))))

    # Compute the ratio
    ratio = p_theta_xz / q_phi_z_given_x_density

    return ratio

def weight(x, theta_hat, theta, A_hat, b_hat, A, b, z):

    
    ## calcul des poids pour theta_hat (associé à l'observation x)
    weight_theta_hat = compute_ratio(x, z, theta_hat, A_hat, b_hat)

    ## cacul des poids pour theta (associé à un theta)                                                                                                    
    weight_theta = compute_ratio(x, z, theta, A, b)
    
    return (weight_theta_hat, weight_theta) ## attention, renvoie un tuple
              
def l_hat(x, theta_hat, theta, A_hat, b_hat, A, b, z_sample): 


    ## calcul de l_theta_hat (pour theta_hat)
    l_theta_hat = np.log((1/(len(z_sample)) * sum(weight(x, theta_hat, theta, A_hat, b_hat, A, b, z_sample[i])[0] for i in range(len(z_sample)))))

    ## calcul de l_theta (pour theta)
    l_theta = np.log((1/(len(z_sample)) * sum(weight(x, theta_hat, theta, A_hat, b_hat, A, b, z_sample[i])[1] for i in range(len(z_sample)))))

    return (l_theta_hat, l_theta)

In [20]:
z_sample[]

(4, 20)

In [19]:
l_hat(z_sample[0], theta_hat, theta_true, A_hat, b_hat, A, b, z_sample)

(-2.832348085865691, -2.785726472660624)

In [5]:
estimateur = Estimateurs.Estimateurs(x, theta_true, 0.6)

In [6]:
estimateur.grad_ML_RR(theta_star=theta_true, n_simulations=100)

ValueError: shapes (4,) and (20,20) not aligned: 4 (dim 0) != 20 (dim 0)